<a href="https://colab.research.google.com/github/Topherrrrr/mec2-projects/blob/main/Korean_Income_and_welfare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving archive.zip to archive.zip


{'archive.zip': b'PK\x03\x04-\x00\x00\x00\x08\x00\xafh\x94Q\x93\xed\x06\x19\xff\xff\xff\xff\xff\xff\xff\xff\x1c\x00\x14\x00Korea Income and Welfare.csv\x01\x00\x10\x00:\x19B\x00\x00\x00\x00\x00\x0f\xcf\x0b\x00\x00\x00\x00\x00\x8c\xbdm\xaep9n\x98\xf9?\x80\xf7P\x0b\xb8)\x88\xd4\xf7j\x8c\x8e\xd3\x134\xc6n\x07\x9dI\x02\xcf\xd6\xe6\xc7,i\xb60G\x12?$\x1e]\x1d\xa1aW\xbdo=\x97WG\xa2$\x92\xa2\xa8\xff\xef\xff\xf9\x7f\xff\xf6_\x7f\xfe\xe3\xaf\x7f\xf9\xc7\xcf\xff\xfe\xcb\xff\xfa\xeb\xcf?\xfe\xfa\xdf\xfe\xf6\xef\x7f\xff\xf9\xdb\xdf\xff\xe5\xdf\xff\xed\xaf?\xff\xc7_\xfe\xedo\xff\xfa\x1f\xff\xfco\x7f\xfd\xb7\xff\xf2\xd7\x7f\xfc\xfc\xb7\xbf\xfe\xfd\xbf>\xffh\xf0?\xff\x97\x7f\xff\xc7\xdf\x7f\xfe\xfa_\xff\xe7\xbf\xfc\xe5\xffz\xf8\x7f\xfe\xd7\xbf\xfe\xaf\xbf\xfe\xeb\xcf\xbf\xfd\xe5\x1f\xff\xf8\xdb_\xfe[\x93\xf2\xaf\x7f\xebr\xfe\xfd_\xfe\xe5\x7f\xfe\xf7\x8e\xfc<\x02\xff\xfb_\xfe\xfe\x1f\xff\xfc?\xfe\xf6\x7f7\xe0/\xff\xe3\xf9\xb1\xbf\xff\xfb\xdf\xff\xfa\xcf\xff\xfb\xdf\xff\xf1\x7f\xfe\xf5\x1f\xff\xf4\x9f\x

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
dataset_name = 'hongsean/korea-income-and-welfare'
zip_name = dataset_name.split('/')[-1]

!kaggle datasets download -d {dataset_name}
!unzip archive.zip -d my_data

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Archive:  archive.zip
  inflating: my_data/Korea Income and Welfare.csv  
  inflating: my_data/job_code_translated.xlsx  


In [ ]:
!pip install scikeras


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
data = pd.read_csv('/content/my_data/Korea Income and Welfare.csv')
data

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,11600.0,6,1,1967,5,1,1,874,1,
92853,98000701,2015,11,5,8327.0,6,1,1967,5,1,1,874,1,
92854,98000701,2016,12,5,7931.0,6,1,1967,5,1,1,874,1,
92855,98000701,2017,13,5,8802.0,5,1,1967,5,1,1,874,1,


In [ ]:
#Removing all null values where the occupation is empty

np.where(pd.isnull(data))
data = data.loc[data['occupation']!=' ']
data = data.fillna(method='ffill')  # Forward fill

data = data.dropna()  # Drop rows with missing values
data = pd.get_dummies(data)

#Scaling the data
scaler = MinMaxScaler()
data['income']=scaler.fit_transform(data[['income']])
data


,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,...,company_size_2,company_size_3,company_size_4,company_size_5,company_size_6,company_size_7,company_size_8,company_size_9,company_size_99,reason_none_worker_
8,20101,2005,1,1,0.446984,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
10,20101,2007,3,1,0.445730,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
11,20101,2008,4,1,0.448353,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
12,20101,2009,5,1,0.447713,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
13,20101,2010,6,1,0.447637,1,2,1945,4,2,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92852,98000701,2014,10,5,0.466789,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92853,98000701,2015,11,5,0.460522,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92854,98000701,2016,12,5,0.459763,6,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1
92855,98000701,2017,13,5,0.461431,5,1,1967,5,1,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
#Separating the X and Y variables. I want to predict the income based on the other values in the dataset

random_state = 436435
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('income', axis=1),
    data['income'], test_size=0.2, random_state=random_state
    )


In [ ]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 47371 entries, 39744 to 72973
Series name: income
Non-Null Count  Dtype  
--------------  -----  
47371 non-null  float64
dtypes: float64(1)
memory usage: 740.2 KB


In [ ]:
X_train.shape

(47371, 266)

In [ ]:
data.columns[:]

Index(['id', 'year', 'wave', 'region', 'income', 'family_member', 'gender',
       'year_born', 'education_level', 'marriage',
       ...
       'company_size_2', 'company_size_3', 'company_size_4', 'company_size_5',
       'company_size_6', 'company_size_7', 'company_size_8', 'company_size_9',
       'company_size_99', 'reason_none_worker_ '],
      dtype='object', length=267)

In [ ]:
type(y_train)

#Scaling the remaining X values in the dataset
continuous_cols = list(data)
continuous_cols.remove('income')
preprocessor = ColumnTransformer(
    transformers = [
        ('cont', MinMaxScaler(), continuous_cols)
    ])
preprocessor.fit_transform(X_train)

array([[0.37476807, 0.23076923, 0.23076923, ..., 0.        , 0.        ,
        0.        ],
       [0.25505049, 0.69230769, 0.69230769, ..., 0.        , 0.        ,
        0.        ],
       [0.07766844, 0.30769231, 0.30769231, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.15431626, 0.69230769, 0.69230769, ..., 0.        , 0.        ,
        0.        ],
       [0.5139793 , 0.23076923, 0.23076923, ..., 0.        , 0.        ,
        0.        ],
       [0.65952954, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
#Creating and training a Sequential model
#It doesn't look like the model is learning anything. My ask is how I could dig deeper into the model to find any underlying issues?

model=Sequential()
model.add(Dense(1, activation='relu', input_shape=(266,)))
model.compile(loss='mae', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1481/1481 [==============================] - 3s 2ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 2/100
1481/1481 [==============================] - 2s 1ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 3/100
1481/1481 [==============================] - 2s 2ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 4/100
1481/1481 [==============================] - 3s 2ms/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 5/100
1481/1481 [==============================] - 1s 980us/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 6/100
1481/1481 [==============================] - 1s 945us/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 7/100
1481/1481 [==============================] - 1s 946us/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 8/100
1481/1481 [==============================] - 1s 966us/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 9/100
1481/1481 [==============================] - 1s 965us/step - loss: 0.4531 - accuracy: 0.0000e+00
Epoch 10/100
1481/1481 [===